In [26]:
import bs4
import os
from dotenv import load_dotenv
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 리트리버


# 답변 준비

In [27]:
# LLM 모델 설정
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print(f"API key exists: {api_key is not None}")

# 웹 페이지 로드
from langchain_community.document_loaders import UnstructuredURLLoader
file_path = ["https://spartacodingclub.kr/blog/all-in-challenge_winner"]
loader = UnstructuredURLLoader(file_path)

# 문서 로드
docs = loader.load()
print(f"로드된 문서 수: {len(docs)}")
print(f"첫 번째 문서 내용 일부: {docs[0].page_content[:100] if docs else 'No content'}")

# 문서 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=10
)
splits = text_splitter.split_documents(docs)
print(f"분할된 청크 수: {len(splits)}")

if len(splits) == 0:
    print("청크가 없어 원본 문서를 사용합니다.")
    splits = docs

# 문서 청크를 Chroma 벡터 스토어에 저장
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))
)

API key exists: True
로드된 문서 수: 1
첫 번째 문서 내용 일부: 

포인트

로딩중

쿠폰

내 강의실

국비 신청 내역

수강권

증명서

숙제 피드백

계정

로그아웃

1725353737651-%C3%A1%C2%84%C2%8F%C3%A1%
분할된 청크 수: 18


INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


self query retriever는 FAISS를 지원하지 않아 Chroma를 이용했습니다.

# 사용자 질문 및 문서 검색

In [28]:
user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."

# 프롬프트 생성

In [29]:
prompt = ChatPromptTemplate.from_template("""<Context>
당신은 공모전 심사위원으로, 검색된 문서 정보를 기반으로 질문에 답변하는 역할입니다.
검색된 문서: {context}
질문: {question}
</Context>

<Objective>
공모전에 대한 유익한 정보를 제공하고, 향후 참가자들이 참고할 수 있는 정보를 정리하는 것입니다.
</Objective>

<Style>
- 검색된 문서 내용을 바탕으로 사실적이고 객관적인 정보만 제공합니다.
- 개인적인 의견이나 해석 없이 검색 결과에서 찾은 정보만 활용합니다.
- 정보를 체계적으로 구조화하여 보기 쉽게 정리합니다.
- 검색 결과에서 찾을 수 없는 정보에 대해서는 명확히 "해당 정보를 제공된 문서에서 찾을 수 없습니다"라고 답변합니다.
</Style>

<Tone>
전문적이고 객관적인 어조로 답변합니다.
</Tone>

<Audience>
향후 이 공모전에 참가할 예정인 잠재적 참가자들입니다.
</Audience>

<Response>
제공된 문서 정보를 체계적으로 분석하여 질문에 대한 정확한 답변을 한국어로 제공하세요. 답변은:
1. 주요 정보를 카테고리별로 명확하게 구분하여 제시
2. 필요시 제목, 소제목, 글머리 기호 등을 활용해 가독성 향상
3. 검색 결과에서 찾은 모든 관련 정보를 포함
4. 답변을 찾을 수 없는 경우 "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 명시
</Response>""")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs), "question": user_msg})
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='<Context>\n당신은 공모전 심사위원으로, 검색된 문서 정보를 기반으로 질문에 답변하는 역할입니다.\n검색된 문서: {context}\n질문: {question}\n</Context>\n\n<Objective>\n공모전에 대한 유익한 정보를 제공하고, 향후 참가자들이 참고할 수 있는 정보를 정리하는 것입니다.\n</Objective>\n\n<Style>\n- 검색된 문서 내용을 바탕으로 사실적이고 객관적인 정보만 제공합니다.\n- 개인적인 의견이나 해석 없이 검색 결과에서 찾은 정보만 활용합니다.\n- 정보를 체계적으로 구조화하여 보기 쉽게 정리합니다.\n- 검색 결과에서 찾을 수 없는 정보에 대해서는 명확히 "해당 정보를 제공된 문서에서 찾을 수 없습니다"라고 답변합니다.\n</Style>\n\n<Tone>\n전문적이고 객관적인 어조로 답변합니다.\n</Tone>\n\n<Audience>\n향후 이 공모전에 참가할 예정인 잠재적 참가자들입니다.\n</Audience>\n\n<Response>\n제공된 문서 정보를 체계적으로 분석하여 질문에 대한 정확한 답변을 한국어로 제공하세요. 답변은:\n1. 주요 정보를 카테고리별로 명확하게 구분하여 제시\n2. 필요시 제목, 소제목, 글머리 기호 등을 활용해 가독성 향상\n3. 검색 결과에서 찾은 모든 관련 정보를 포함\n4. 답변을 찾을 수 없는 경우 "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 명시\n</Response>'), additional_kwargs={})]


LLM에게 전달할 프롬프트 템플릿을 생성합니다. 프롬프트는 공모전 심사위원 역할을 부여하고, 검색된 문서의 정보만을 활용하여 객관적인 정보를 제공하도록 지시합니다.

사용자 질문을 정의하고, 해당 질문에 관련된 문서를 리트리버를 통해 검색합니다. 검색된 문서의 수를 출력하고, 검색된 문서 목록을 확인합니다.

### 1. similarity 리트리버

In [30]:
retriever_similarity = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 50})
retrieved_docs_similarity = retriever_similarity.invoke(user_msg)

print(retrieved_docs_similarity)

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[Document(id='73d27d4e-4044-4a3f-bb95-fc836bea78f6', metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='조회수 756·6분 분량\n\n2024. 9. 3.\n\n코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.\n\n<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.\n\n🏆 대상\n\n[Lexi Note] 언어공부 필기 웹 서비스\n\n서비스 제작자: 다나와(김다애, 박나경)'), Document(id='132f2451-ed24-4f61-8368-ff8972b4bdfe', metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='조회수 756·6분 분량\n\n2024. 9. 3.\n\n코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.\n\n<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로

### 2. mmr 리트리버
MMR은 관련성과 다양성을 모두 고려하여 검색 결과의 중복을 줄이고 다양한 정보를 제공하는 데 유용합니다.

In [31]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

retriever_mmr = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 50})
retrieved_docs_mmr = retriever_mmr.invoke(user_msg)

print(retrieved_docs_mmr)

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[Document(id='73d27d4e-4044-4a3f-bb95-fc836bea78f6', metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='조회수 756·6분 분량\n\n2024. 9. 3.\n\n코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.\n\n<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.\n\n🏆 대상\n\n[Lexi Note] 언어공부 필기 웹 서비스\n\n서비스 제작자: 다나와(김다애, 박나경)'), Document(id='132f2451-ed24-4f61-8368-ff8972b4bdfe', metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='조회수 756·6분 분량\n\n2024. 9. 3.\n\n코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.\n\n<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로

### 3. 문서압축 리트리버
LLM을 이용해 검색된 문서를 압축하여 질문과 관련된 핵심 내용만 추출합니다.

In [33]:
base_retriever_context = vectorstore.as_retriever(
                                search_type='similarity',
                                search_kwargs={'k':50})
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=base_retriever_context
)

compressed_docs_context = compression_retriever.get_relevant_documents(user_msg)
print(len(compressed_docs_context))
compressed_docs_context

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/comp

17


[Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.\n\n🏆 대상\n\n[Lexi Note] 언어공부 필기 웹 서비스\n\n서비스 제작자: 다나와(김다애, 박나경)'),
 Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.\n\n🏆 대상\n\n[Lexi Note] 언어공부 필기 웹 서비스\n\n서비스 제작자: 다나와(김다애, 박나경)'),
 Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.\n\n🏆 대상\n\n[Lexi Note] 언어공부 필기 웹 서비스\n\n서비스 제작자: 다나와(김다애, 박나경)'),
 Document(metadata={'source': 'https://spartacodingc

### 4. self-query 리트리버
이 방법은 메타데이터 필드를 정의하여 LLM이 사용자 질문을 구조화된 쿼리로 변환하도록 합니다. 공모전과 관련된 메타데이터(상장 종류, 공모전 이름, 수상자 등)를 정의하여 보다 정확한 검색이 가능하도록 합니다.

In [34]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# 수상 정보 정의
metadata_field_info = [
    AttributeInfo(
        name="award_type",
        description="상장 종류 (대상, 최우수상, 우수상, 입선선 등)",
        type="string",
    ),
    AttributeInfo(
        name="competition_name", 
        description="공모전 이름", 
        type="string"
    ),
    AttributeInfo(
        name="recipients_name", 
        description="수상자들의의 이름", 
        type="string"
    ),
    AttributeInfo(
        name="recipient_work_decription", 
        description="수상작에 대한 설명", 
        type="string"
    ),
    AttributeInfo(
        name="tech_stack", 
        description="사용한 기술", 
        type="string"
    ),
]

self_query_retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_contents="ALL-in 코딩 공모전 수상작 정보",
    metadata_field_info=metadata_field_info,
    enable_limit=True,
    search_kwargs={"k": 50},  # k 의 값을 2로 지정하여 검색 결과를 2개로 제한합니다.
)

retrieved_docs_self_query = self_query_retriever.get_relevant_documents(user_msg)
print(len(retrieved_docs_self_query))
retrieved_docs_self_query


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


50


[Document(id='bd3d7b56-2b12-4853-b476-d917c3397ed5', metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='FE(프론트엔드): React, Tesseract.js, React-Quill, HTML, CSS, JavaScript\n\nBE(백엔드): Java, Spring Boot, MariaDB\n\n🎖️ 우수상\n\n[우리집 히어로즈] 벌레 퇴치 영웅 매칭 서비스\n\n서비스 제작자: 인트(배정연, 한지수)\n\n💡C는 이번 학기 처음으로 자취를 시작했습니다. 원래부터 작은 벌레에 민감했던 C. 자취방에서 자주 나타나는 벌레들 때문에 밤잠을 설치길 반복했습니다. 밤마다 벌레를 잡으려고 애쓰던 C는 커뮤니티를 통해 다른 학생들도 같은 문제를 겪고 있다는 것을 알게 되었습니다. 하지만 커뮤니티에서 벌레 퇴치 히어로를 찾기 위해서는 내 개인 정보를 노출해야 한다는 점이 찝찝했죠.'),
 Document(id='d9e37fe3-0262-4ea1-9970-daa50023b13d', metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='FE(프론트엔드): React, Tesseract.js, React-Quill, HTML, CSS, JavaScript\n\nBE(백엔드): Java, Spring Boot, MariaDB\n\n🎖️ 우수상\n\n[우리집 히어로즈] 벌레 퇴치 영웅 매칭 서비스\n\n서비스 제작자: 인트(배정연, 한지수)\n\n💡C는 이번 학기 처음으로 자취를 시작했습니다. 원래부터 작은 벌레에 민감했던 C. 자취방에서 자주 나타나는 벌레들 때문에 밤잠을 설치길 반복했습니다. 밤마다 벌레를 잡으려고 애쓰던 C는 커뮤니티를 통해 다른 학생들도 같은 문제를 겪고 있다는 것을 알게 되었습니다

리트리버들의 결과 비교

In [35]:
print("1. search type: similarity")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs_similarity), "question": user_msg})
response = llm.invoke(user_prompt)
print(response.content)


print("\n\n\n2. search type: mmr")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs_mmr), "question": user_msg})
response = llm.invoke(user_prompt)


print(response.content)
print("\n\n\n3. contextual compression retriever")
user_prompt = prompt.invoke({"context": format_docs(compressed_docs_context), "question": user_msg})
response = llm.invoke(user_prompt)


print(response.content)
print("\n\n\n4. self-query retriever")
user_prompt = prompt.invoke({"context": format_docs(retrieved_docs_self_query), "question": user_msg})
response = llm.invoke(user_prompt)
print(response.content)


1. search type: similarity


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


# All-in 코딩 공모전 수상작 요약

## 공모전 개요
- **주최**: 스파르타코딩클럽
- **목적**: 대학생들이 캠퍼스에서 겪은 불편함과 문제를 해결하기 위한 자신만의 아이디어를 구현
- **결과**: 다양한 혁신적인 아이디어로 가득한 여러 프로젝트가 출품됨

## 수상작 소개

### 1. 대상 수상작
- **제목**: [Lexi Note] 언어공부 필기 웹 서비스
  - **제작자**: 다나와(김다애, 박나경)
  - **서비스 개요**: 단어를 드래그하면 네이버 사전과 연동돼 단어의 의미를 찾고, 번역 기능을 통해 긴 문장을 쉽게 이해할 수 있는 기능 제공. 할 일 목록과 스케줄 템플릿도 포함되어 있어 효율적인 언어 학습 도우미 역할을 함.
  - **사용 기술 스택**: 
    - 프론트엔드: Spring Boot, MySQL, Spring WebSocket
    - 백엔드: React Native, TanStack Query, Axios

### 2. 우수상 수상작
- **제목**: [우리집 히어로즈] 벌레 퇴치 영웅 매칭 서비스
  - **제작자**: 인트(배정연, 한지수)
  - **서비스 개요**: 자취생들이 자취방에서 겪는 벌레 문제를 해결하기 위해 매칭 서비스를 제공. 벌레 퇴치가 필요한 사용자가 요청을 올리면, 같은 학교 학생이 퇴치 작업을 대신해 주는 시스템.
  - **사용 기술 스택**:
    - 프론트엔드: React, Tesseract.js, React-Quill, HTML, CSS, JavaScript
    - 백엔드: Java, Spring Boot, MariaDB

### 3. 입선작
- **제목**: [Crewing] 연합동아리 정보 플랫폼
  - **제작자**: 동학대학운동(김민아, 임경진, 신은혜, 고수)
  - **서비스 개요**: 대학생들의 연합 동아리 가입을 지원하는 아카이빙 플랫폼으로, 개인 정보를 입력하면 적절한 동아리를 추천받고, 서류 전형 및 인터뷰 과정을 효율적으로 관리 가능.
  - **

INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### ALL-in 코딩 공모전 수상작 요약

#### 공모전 개요
- **주최**: 스파르타코딩클럽
- **목적**: 대학생들이 캠퍼스에서 겪는 불편함과 문제를 아이디어로 해결하는 플랫폼 제공
- **대상**: 대학생
- **참여 방안**: 코딩 및 창의력을 통해 자신의 아이디어 실현

#### 수상작 소개
이번 공모전에서 뛰어난 성과를 보인 수상작 6개 중 **대상** 수상작은 다음과 같습니다.

1. **수상작**: [Lexi Note] 언어공부 필기 웹 서비스  
   - **제작자**: 다나와 (김다애, 박나경)

#### 공모전의 특징
- **창의성 강조**: 누구나 아이디어를 바탕으로 서비스를 개발할 수 있도록 지원
- **기술적 지원**: 스파르타코딩클럽에서 비전공자와 초보자 대상의 다양한 개발 프로그램 제공
- **팀 프로젝트**: 참가자들은 팀을 구성하여 협력하여 프로젝트를 발전시킴

#### 기술 스택
- **프론트엔드**: Spring Boot, MySQL, Spring WebSocket
- **백엔드**: React Native, TanStack Query, Axios

#### 참가자 메시지
공모전 참가자들은 자신들의 아이디어가 실제로 문제를 해결하는 과정을 경험하며, 이를 통해 커다란 성과를 도출할 수 있음을 시사합니다.

> 더 많은 참가자의 문제 해결 아이디어와 결과물은 'All-in 공모전' 플랫폼에서 확인할 수 있습니다.

#### 결론
ALL-in 코딩 공모전은 창의적인 아이디어를 바탕으로 실질적인 문제를 해결하는 기회를 제공하며, 미래의 개발자들에게 영감을 주는 중요한 행사입니다. 향후 참가자들은 이러한 수상작을 참고하여 자신의 프로젝트 아이디어를 개발하는 데 유익한 자료로 활용할 수 있습니다.



3. contextual compression retriever


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### ALL-in 코딩 공모전 수상작 요약

#### 1. 공모전 개요
- **공모전 이름**: ALL-in 코딩 공모전
- **목적**: 대학생들이 캠퍼스에서 겪는 불편함과 문제를 창의적 아이디어로 해결하기 위한 대회
- **참여 형태**: 팀 프로젝트를 통해 개발된 웹/앱 서비스

#### 2. 수상작 소개
1. **대상**
   - **프로젝트 이름**: Lexi Note
   - **서비스 유형**: 언어공부 필기 웹 서비스
   - **제작자**: 다나와 (김다애, 박나경)

2. **우수상**
   - **프로젝트 이름**: 우리집 히어로즈
     - **서비스 유형**: 벌레 퇴치 영웅 매칭 서비스
     - **제작자**: 인트 (배정연, 한지수)
     - **문제 인사이트**: 자취생들이 겪는 벌레 문제와 개인 정보 노출의 우려 해결

   - **프로젝트 이름**: 에코 클래스룸
     - **서비스 유형**: 수업 실시간 소통 서비스
     - **문제 인사이트**: 수업 중 학생들과의 즉각적인 소통 필요성 해결

### 결론
ALL-in 코딩 공모전은 대학생들이 현실의 문제를 해결하기 위한 혁신적인 아이디어를 실현할 수 있는 기회를 제공합니다. 이번 대회에서는 특히 독창적인 아이디어와 열정적으로 개발된 프로젝트들이 돋보였습니다. 향후 참가자들에게는 이러한 다양한 프로젝트가 참고가 될 것입니다.



4. self-query retriever


INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


### ALL-in 코딩 공모전 수상작 요약

#### 🎖️ 우수상 수상작

1. **우리집 히어로즈**
   - **제작자:** 인트(배정연, 한지수)
   - **서비스 개요:**
     - 자취생들이 자취방에서 겪는 벌레 문제를 해결하기 위해 개발된 앱.
     - 벌레 퇴치가 필요한 사용자가 요청을 올리면 같은 학교 학생인 벌레 퇴치 히어로와 매칭됨.
     - 사용자 신원 보장으로 안전한 환경에서 서비스 제공.
     - 실시간 알림 기능을 통해 빠른 문제 해결 가능.
     - 사용자 간 평가 시스템을 통해 서비스 품질 향상.

   - **기술 스택:**
     - **프론트엔드:** React Native, Expo, Axios
     - **백엔드:** Spring Boot, Spring Security, MySQL, SSE, FCM, JWT, Postman

---

2. **에코 클래스룸**
   - **제작자:** (정보 없음)
   - **서비스 관련 정보:** 제공된 문서에서 해당 정보를 찾을 수 없습니다.
   - **기술 스택:** 제공된 문서에서 해당 정보를 찾을 수 없습니다.

---

#### 🏅 입선 수상작

1. **BLOTIE**
   - **제작자:** 블로티(이은주, 한명수, 황준영)
   - **서비스 개요:**
     - 교내 외국인과 내국인 매칭 및 교류를 지원하는 플랫폼.
     - 외국인 학생들의 한국 적응을 돕기 위한 기능 제공.

   - **기술 스택:** 제공된 문서에서 해당 정보를 찾을 수 없습니다.

---

2. **Crewing**
   - **제작자:** 동학대학운동(김민아, 임경진, 신은혜, 고수)
   - **서비스 개요:**
     - 대학생들이 다양한 연합 동아리에 쉽게 가입하고, 적절한 동아리를 찾을 수 있도록 지원하는 플랫폼.
     - 회원가입 시 개인 정보를 입력하여 맞춤형 동아리 추천.
     - 동아리 리크루팅 과정의 효율적 관리 제공 (서류 전형, 인터

Self-Query 리트리버가 가장 효과적인 결과를 보여주었습니다. 이 방식은 메타데이터 필드(상장 종류, 공모전 이름, 수상자 등)를 명확히 정의하여 LLM이 사용자 질문을 구조화된 쿼리로 변환할 수 있게 했습니다. 그 결과, 다른 리트리버들이 놓친 "BLOTIE" 입선작까지 포함한 더 포괄적인 정보를 제공했고, 프론트엔드/백엔드 기술 스택 구분, 서비스 기능 설명, 문제 해결 방식 등 더 체계적인 정보를 추출할 수 있었습니다. 특히 특정 속성에 대한 정보를 정확하게 추출해야 하는 상황에서 Self-Query 리트리버의 강점이 두드러졌으며, 이는 복잡한 정보를 구조화하여 검색하는 데 효과적인 접근법임을 보여줍니다.